# 01 准确度的陷阱和混淆矩阵

## 分类准确度问题
一个癌症预测系统，输入体检信息，可以判断是否有癌症，如果一个系统的预测准确度是99.9%，而癌症的概率只有0.01%，那预测所有人都是健康的，可以达到99.99%的准确率，即这个预测系统是失败的

如此产生的原因是：对于极度偏斜(Skewed Data)的数据，只使用分类准确度是远远不够的

使用混淆矩阵做进一步的分析

## 混淆矩阵 (Confusion matrix)
对于二分类问题
- 预测
    - 0-Ngeative
    - 1-Positive
- 真实
    - 0-False
    - 1-True

xx:预测结果预测值


|真实值\\预测值|0|1|
|:----:|:----:|:----:|
|0|TN|FP|
|1|FN|TP|


# 02 精准率和召回率

在癌症预测系统中对于10000个病人进行分类，混淆矩阵如下：

|真实值\\预测值|0|1|
|:----:|:----:|:----:|
|0|9978|12|
|1|2|8|

精准率(预测为1的正确数在预测为1中的占比):$precision = \frac{TP}{TP+FP}$

精准率 = 8/(8+12) = 40%

召回率(预测为1的正确数在真实为1中的占比):$recall = \frac{TP}{TP+FN}$

召回率 = 8/(8+2) = 80%

# 03 代码实现

In [1]:
import numpy as np
from sklearn import datasets

In [2]:
digits = datasets.load_digits()
X = digits.data
y = digits.target.copy()

# 构造偏斜数据
y[digits.target == 9] = 1
y[digits.target != 9] = 0

In [3]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

In [4]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver='liblinear') # 可以消除warining
log_reg.fit(X_train,y_train)
log_reg.score(X_test,y_test)

0.9755555555555555

In [5]:
y_log_predict = log_reg.predict(X_test)

In [6]:
def TN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 0) & (y_predict == 0))
TN(y_test,y_log_predict)

403

In [7]:
def FP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 0) & (y_predict == 1))
FP(y_test,y_log_predict)

2

In [8]:
def FN(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 1) & (y_predict == 0))
FN(y_test,y_log_prodict)

9

In [9]:
def TP(y_true, y_predict):
    assert len(y_true) == len(y_predict)
    return np.sum((y_true == 1) & (y_predict == 1))
TP(y_test,y_log_predict)

36

In [10]:
def confusion_matrix(y_true, y_predict):
    return np.array([
        [TN(y_true, y_predict), FP(y_true, y_predict)],
        [FN(y_true, y_predict), TP(y_true, y_predict)]
    ])

confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]])

In [11]:
def precision_score(y_true, y_predict):
    tp = TP(y_true, y_predict)
    fp = FP(y_true, y_predict)
    try:
        return tp / (tp + fp)
    except:
        return 0.0
precision_score(y_test, y_log_predict)

0.9473684210526315

In [12]:
def recall_score(y_true, y_predict):
    tp = TP(y_true, y_predict)
    fn = FN(y_true, y_predict)
    try:
        return tp / (tp + fn)
    except:
        return 0.0
recall_score(y_test, y_log_predict)

0.8

###  在sklearn 中

In [13]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_log_predict)

array([[403,   2],
       [  9,  36]])

In [14]:
from sklearn.metrics import precision_score

precision_score(y_test, y_log_predict)

0.9473684210526315

In [15]:
from sklearn.metrics import recall_score

recall_score(y_test, y_log_predict)

0.8

# F1 Score

F1，是精准率和召回率的调和平均数，兼顾精准率和召回率

$$F1 = \frac{2 \cdot precision \cdot recall}{precision + recall}$$

In [1]:
import numpy as np
from sklearn import datasets

In [2]:
def f1_score(precision, recall):
    try:
        return 2 * (precision * recall) / (precision + recall)
    except:
        return 0.0

In [3]:
precision = 0.1
recall = 0.9
f1_score(precision, recall)

0.18000000000000002

In [4]:
digits = datasets.load_digits()
X = digits.data
y = digits.target.copy()

# 构造偏斜数据
y[digits.target == 9] = 1
y[digits.target != 9] = 0

In [5]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=666)

In [6]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression(solver='liblinear') # 可以消除warining
log_reg.fit(X_train,y_train)
log_reg.score(X_test,y_test)

0.9755555555555555

In [7]:
y_predict = log_reg.predict(X_test)

In [10]:
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_predict)

array([[403,   2],
       [  9,  36]])

In [12]:
from sklearn.metrics import precision_score

precision_score(y_test, y_predict)

0.9473684210526315

In [13]:
from sklearn.metrics import recall_score

recall_score(y_test, y_predict)

0.8

In [14]:
from sklearn.metrics import f1_score

f1_score(y_test, y_predict)

0.8674698795180723